In [2]:
### Load libraries
library(here)
library(rgeos)
library(geosphere)
library(rgdal)
library("ggplot2")
library("tidyr")
library(raster) # for crs
library(ggbeeswarm)
library(tmap)
library(sf)
library(stringr)
library(scales)
library("ggrepel")

In [3]:
### Load utility functions and themes
source(here("R","getNicheOutline.R"))
source(here("R","theme_tmaps.R"))

In [4]:
### Define paths

# Define imputs paths:

# Admnistrative map of China
path2china<-here('raw_data','CHN_adm')
path2out<-here("outputs")

# Archaeological records location data
path2buckwheat<-here("data","ESM_2.csv") # Buckwheat's occurrence records
path2locations<-here("data","site_locations.csv")

# Production and predictions data
path2predictions <- here("outputs","predictions")

In [5]:
# Define output paths:

path2outputs<-here("manuscript","Chapter6","Figs","Raster")

In [6]:
# Import data
loc<-read.csv(path2locations)
ord<-as.numeric(gsub(" BP","",unique(loc$period)))
loc$period<-factor(loc$period,levels=unique(loc$period)[order(ord)],ordered=TRUE)
coordinates(loc)<-~Longitude+Latitude

In [7]:
# Import the outer border of China
outer_border<-st_read(dsn=path2china,layer='CHN_adm0')
# Import the spatial polygons data frame of counties:
china<-readOGR(dsn=path2china,layer='CHN_adm3')
# Import production data

Reading layer `CHN_adm0' from data source `G:\My Drive\my_repositories\PhD\raw_data\CHN_adm' using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 70 fields
geometry type:  MULTIPOLYGON
dimension:      XY
bbox:           xmin: 73.5577 ymin: 18.15931 xmax: 134.7739 ymax: 53.56086
geographic CRS: WGS 84
OGR data source with driver: ESRI Shapefile 
Source: "G:\My Drive\my_repositories\PhD\raw_data\CHN_adm", layer: "CHN_adm3"
with 2409 features
It has 13 fields
Integer64 fields read as strings:  ID_0 ID_1 ID_2 ID_3 


In [8]:
### Transform china spatial points dataframe to simple features
china_sf<-st_as_sf(china) 

In [9]:
# List of crops in the analysis:

crops<-c("rice","millet","buckwheat","wheat","barley")

In [10]:
for (crop in crops){
    print(crop)
    path2pred_summary<-paste(path2predictions,"/",crop,"_predictions.csv",sep="")
    pred_summary<-read.csv(path2pred_summary,row.names=1)

    ### Get the log means predictions, 0.05, and 0.95 quantiles for each county in a separate variables
    mean_names<-colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("mean",colnames(pred_summary))]
    mean<-pred_summary[,colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("mean",colnames(pred_summary))]]
    q5_names<-colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("X5",colnames(pred_summary))]
    q5<-pred_summary[,colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("X5",colnames(pred_summary))]]
    q95_names<-colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("X95",colnames(pred_summary))]
    q95<-pred_summary[,colnames(pred_summary)[grepl("log",colnames(pred_summary))&grepl("X95",colnames(pred_summary))]]

    # Put the values together in the dataframe
    df<-cbind(mean,q5,q95)
    colnames(df)<-paste(crop,colnames(df),sep="_")
    # Simplify colum names a bit
    colnames(df)<-gsub("s_pred","",colnames(df))
    colnames(df)<-gsub("_X","",colnames(df))
    colnames(df)<-gsub("\\.","",colnames(df))
    
    # Bind data to spatial polygons
    china@data<-cbind(china@data,df)
    }

[1] "rice"
[1] "millet"
[1] "buckwheat"
[1] "wheat"
[1] "barley"


In [11]:
### Bind the location data, with the predictions for counties in which buckwheat was found:
proj4string(loc) <- crs(china)
pred_data<-sp::over(loc,china)
loc@data<-cbind(loc@data,pred_data)

In [12]:
crops<-c("rice","millet","buckwheat","wheat","barley")

In [13]:
for (crop in (crops)){
    columns<-grep(paste("^",crop,sep=""),tolower(colnames(loc@data)))
    if(crop%in%c("millet","buckwheat")){columns<-grep(crop,tolower(colnames(loc@data)))}
    columns<-columns[!(columns%in%grep("_log",tolower(colnames(loc@data))))]
    for (column in columns){
        timeslices<-unique(loc@data[!is.na(loc@data[column]),]$period)
        for (i in timeslices){
            print(crop)
            print(column)
            print(i)
            loc@data[loc@data$period==i&!is.na(loc@data[,column]),paste(crop,c('Mean','5','95'),sep="")]<- loc@data[loc@data$period==i&!is.na(loc@data[,column]),colnames(loc@data) %in% paste(crop,c("_mean","5","95"),"_",gsub(" BP","",i),"_log",sep="")]
        }
    }
}

[1] "rice"
[1] 10
[1] "3000 BP"
[1] "rice"
[1] 10
[1] "2000 BP"
[1] "rice"
[1] 10
[1] "4000 BP"
[1] "rice"
[1] 10
[1] "7000 BP"
[1] "rice"
[1] 10
[1] "5000 BP"
[1] "rice"
[1] 10
[1] "8000 BP"
[1] "rice"
[1] 10
[1] "6000 BP"
[1] "rice"
[1] 10
[1] "9000 BP"
[1] "rice"
[1] 10
[1] "1000 BP"
[1] "rice"
[1] 10
[1] "10000 BP"
[1] "rice"
[1] 10
[1] "14000 BP"
[1] "rice"
[1] 10
[1] "17000 BP"
[1] "rice"
[1] 11
[1] "12000 BP"
[1] "rice"
[1] 11
[1] "14000 BP"
[1] "rice"
[1] 11
[1] "17000 BP"
[1] "millet"
[1] 7
[1] "4000 BP"
[1] "millet"
[1] 7
[1] "2000 BP"
[1] "millet"
[1] 7
[1] "3000 BP"
[1] "millet"
[1] 7
[1] "5000 BP"
[1] "millet"
[1] 7
[1] "6000 BP"
[1] "millet"
[1] 7
[1] "8000 BP"
[1] "millet"
[1] 7
[1] "7000 BP"
[1] "millet"
[1] 7
[1] "1000 BP"
[1] "millet"
[1] 7
[1] "9000 BP"
[1] "millet"
[1] 8
[1] "4000 BP"
[1] "millet"
[1] 8
[1] "2000 BP"
[1] "millet"
[1] 8
[1] "3000 BP"
[1] "millet"
[1] 8
[1] "5000 BP"
[1] "millet"
[1] 8
[1] "7000 BP"
[1] "millet"
[1] 8
[1] "6000 BP"
[1] "millet"
[1] 8


In [16]:
write.csv(loc@data,file=here("data","locations_suitability.csv"),row.names=FALSE)

In [21]:
write.csv(as.data.frame(colnames(china@data)),here("data","full_names.csv"))

In [ ]:
writeOGR(obj=china, dsn=here("data","predictions"), layer="predictions", driver="ESRI Shapefile") # this is in geographical projection


In [16]:
# Get info about country's area 
china@data$area <- areaPolygon(china)

In [31]:
#for (crop in crops){
    crop="buckwheat"
    print(crop)
        
    threshold<-min(loc@data[,paste(crop,"Mean",sep="")],na.rm=TRUE)

    if(crop%in%c("wheat","barley")){

        macros<-loc@data[,c("Site","period",paste(crop,"Mean",sep=""))]
        colnames(macros)<-sub(paste(crop,"Mean",sep=""),"mean",colnames(macros))
        macros<-macros[!is.na(macros$mean),]
        
        lower_threshold<-threshold
        if(crop=="wheat"){
            threshold<-unique(sort(macros$mean))[3]
            site_labels<-rbind(unique(macros[macros$mean%in%unique(sort(macros$mean))[1:6],c("Site","period","mean")]),unique(macros[macros$period=="5000 BP",c("Site","period","mean")]),
                               macros[macros$period %in% c("4000 BP")&(macros$Site %in% loc[(coordinates(loc)[,"Latitude"]>40& coordinates(loc)[,"Longitude"]<90),]@data$Site),],
                                                             macros[macros$period %in% c("4000 BP")&macros$mean==min(macros[macros$period %in% c("4000 BP"),]$mean),])}
        else{
            threshold<-unique(sort(macros$mean))[4]
            site_labels<-rbind(unique(macros[macros$mean%in%unique(sort(macros$mean))[1:5],c("Site","period","mean")]),unique(macros[macros$period=="5000 BP",c("Site","period","mean")]))}
        rf<-0.1
    }else{
            macros<-loc@data[,c("Site","period","Sample_type",paste(crop,"Mean",sep=""))]
            macros<-cbind(macros,loc@data[grep(crop,tolower(colnames(loc@data)[1:12]))])
            rf<-1
            if(crop=="buckwheat"){colnames(macros)[5:6]<-c("common\nbuckwheat","tartary\nbuckwheat")}
            else if(crop=="rice"){colnames(macros)[5:6]<-c("rice","wild\nrice")}
            colnames(macros)<-sub(paste(crop,"Mean",sep=""),"mean",colnames(macros))
            macros<-macros[!is.na(macros$mean),]
            if(crop=="buckwheat"){macros<-gather(macros,species,present,-Site,-period,-mean,-Sample_type)
                                  macros<-macros[!is.na(macros$present),names(macros)!="present"]                                  
                                  macros$species<-ifelse(macros$specie=="common\nbuckwheat",macros$Sample_type,macros$species)
                                  macros$species<-tolower(sub(" ","\n",macros$species))}
            else if (crop=="millet"){macros$species<-ifelse(!is.na(macros$Foxtail.millet)&!is.na(macros$Broomcorn.millet),"both",
                        ifelse(!is.na(macros$Foxtail.millet),"foxtail",
                              ifelse(!is.na(macros$Broomcorn.millet),"broomcron","unidentified")))
                        # Get rid of double Xinlongwa/Xinlonggou    
                        macros<-macros[macros$Site!="Xinglonggou",]
                                    }
               
        
            if(crop=="rice"){
                macros<-gather(macros,species,present,-Site,-period,-mean,-Sample_type)
                macros<-macros[!is.na(macros$present),names(macros)!="present"] 
                # Adjust wild species for early periods
                macros$species<-ifelse(macros$period>"10000 BP","wild\nrice",macros$species)
                site_labels<-rbind(unique(macros[macros$mean%in%unique(sort(macros$mean))[1],c("Site","period","mean")],),
                                   unique(macros[macros$period>"8000 BP",c("Site","period","mean")]),
                                   macros[(macros$period %in% c("7000 BP","8000 BP")&macros$Site %in% loc[coordinates(loc)[,"Latitude"]>33.6,]@data$Site),c("Site","period","mean")],
                                   macros[(macros$period %in% c("4000 BP")&macros$Site %in% loc[coordinates(loc)[,"Latitude"]>39,]@data$Site),c("Site","period","mean")],
                                   macros[(macros$period == "6000 BP")&macros$Site %in% loc[coordinates(loc)[,"Latitude"]<28,]@data$Site,c("Site","period","mean")])

               # site_labels<-rbind(unique(macros[macros$mean%in%unique(sort(macros$mean))[1],c("Site","period","mean")],),unique(macros[macros$period>"8000 BP",c("Site","period","mean")]))
                site_labels$Site<-gsub(" ","\n",site_labels$Site)
            }
            else if(crop=="millet"){lower_threshold<-threshold
                           threshold<-unique(sort(macros$mean))[4]
                           site_labels<-rbind(unique(macros[macros$mean%in%unique(sort(macros$mean))[1:4],c("Site","period","mean")]),
                                              unique(macros[macros$period>="8000 BP",c("Site","period","mean")]),
                                              macros[(macros$period %in% c("4000 BP")&macros$Site %in% loc[coordinates(loc)[,"Latitude"]<25,]@data$Site),c("Site","period","mean")],
                                              macros[(macros$Site %in% loc[coordinates(loc)[,"Latitude"]>40&coordinates(loc)[,"Longitude"]>110,]@data$Site),c("Site","period","mean")],
                                             macros[(macros$Site %in% loc[coordinates(loc)[,"Latitude"]>40&coordinates(loc)[,"Longitude"]<90,]@data$Site),c("Site","period","mean")])}
            else if(crop=="buckwheat"){
                site_labels<-rbind(unique(macros[macros$species=="tartary\nbuckwheat",c("Site","period","mean")]),unique(macros[macros$period>="6000 BP",c("Site","period","mean")]))  
                threshold<-min(loc@data[!is.na(loc@data$Buckwheat),paste(crop,"Mean",sep="")],na.rm=TRUE)}
        }
    
    predByPeriod <- gather(china@data,period,predictions,grep(paste("^",crop,"_mean",sep=""),colnames(china@data)))
    d<-predByPeriod[,c('predictions','period','area')]
    
    d$period<-gsub(paste(crop,"mean_",sep="_"),"",d$period)
    d$period<-sub("_log"," BP",d$period)
    
    d<-d[as.numeric(gsub(" BP", "",d$period))<=max(as.numeric(gsub(" BP", "",macros$period))),]
    d$period <- factor(d$period,levels = unique(d$period),ordered = TRUE)
    
    # For scales
    min=as.numeric(gsub(".*e","1e",as.character(scientific(exp(min(d$predictions,na.rm=TRUE))))))
    max=as.numeric(gsub(".*e","1e",as.character(scientific(exp(max(d$predictions,na.rm=TRUE))))))
    breaks<-signif(exp(seq(log(min), log(max), length.out = log10(max/min)+1)),1)
        
    ### Prepare locations data Set the same levels
    # Set the same levels
    macros$period<-factor(macros$period,levels = unique(d$period),ordered = TRUE)
    
    violin<-plot<-ggplot()+
    geom_violin(data=d, aes(x=factor(period), y=exp(predictions),weight=area),color="#6497b1",fill="#b3cde0")+
    geom_boxplot(data=d, aes(x=period, y=exp(predictions),weight=area),col="#005b96",width=0.1,outlier.shape=NA)+
    scale_y_continuous(trans='log2',breaks=breaks[c(seq(length(breaks), 1, by=-5))])+
    theme_classic()+
    labs(y = "PREDICTED VALUE\n(proportion of the area under cultivation)\n")+
    theme(
      axis.title.y=element_text(size=6,face="bold"),
      axis.text=element_text(size=7),
      axis.title.x=element_blank(),
      legend.position= c(0.5, 0.1),
      legend.box = "horizontal",
      legend.direction="horizontal",
      legend.title = element_blank(),
      legend.text = element_text(size = 6),
      legend.key.width = unit(0.5,"cm"),
      legend.spacing.x = unit(0.05,'cm'))+
    guides(colour = guide_legend(nrow = 1))
    
    if (!(crop %in% c("rice","buckwheat"))){
        violin<-violin+
            geom_hline(aes(yintercept = c(exp(lower_threshold),exp(threshold)),colour=c(paste("minimum predicted value for sites\nwhere",crop,"remains were found"),"threshold without outliers")),size=0.4,linetype=3)+   
            scale_color_manual(values=c("mediumorchid4","red"))+
            geom_text_repel(data=site_labels,force=rf,aes(x=period,y=exp(mean),label=Site),cex=1.5,fontface="bold")
    }else{violin<-violin+geom_hline(aes(yintercept = exp(threshold),colour=paste("minimum predicted value for sites\nwhere",crop,"remains were found")),linetype=3,size=0.4)+   
                            scale_colour_manual(values="red")}
    
    if (crop=="rice"){
        violin<-violin+
            geom_point(data=macros[!is.na(macros$period),],aes(x=factor(period), y=exp(mean),shape=species,fill=species,cex=species),alpha=0.8,position=position_beeswarm(cex=0.6))+
            scale_fill_manual(name="",values=c("black","mediumseagreen"),drop=FALSE)+ 
            scale_shape_manual(name="",values=c(21,22),drop=FALSE)+
            scale_size_manual(name="",values=c(0.8,1.2),drop=FALSE)+
            geom_text_repel(data=site_labels,force=1,aes(x=period,y=exp(mean),label=Site),cex=1.7,fontface="bold")
}else if (crop=="millet"){
        violin<-violin+
            geom_point(data=macros[!is.na(macros$period),],aes(x=factor(period), y=exp(mean),shape=species,fill=species),cex=0.7,alpha=0.8,position=position_beeswarm(cex=0.6))+
            scale_fill_manual(name="",values=c("maroon2","forestgreen","orange","white"),drop=FALSE)+ 
            scale_shape_manual(name="",values=c(21,24,22,24),drop=FALSE)
    }else if(crop=="buckwheat"){
        violin<-violin+ geom_point(data=macros[!is.na(macros$period),],aes(x=factor(period), y=exp(mean),shape=species,fill=species),cex=1,alpha=0.8,position= position_beeswarm())+
        scale_fill_manual(name="",values=c("black","mediumseagreen","orange","red"),drop=FALSE)+ 
        scale_shape_manual(name="",values=c(24,21,22,25),drop=FALSE)+
        geom_text(data=site_labels,position = position_nudge(0.25,-1),aes(x=period,y=exp(mean),label=Site),cex=1.5,fontface="bold")
    }else{
        violin<-violin+geom_point(data=macros[!is.na(macros$period),],aes(x=factor(period), y=exp(mean),shape=paste("predictions for the locations\nof",crop,"records")),fill="black",cex=0.5,alpha=0.8,position=position_beeswarm(cex=0.6))+
        scale_shape_manual(name="",values=c(21),drop=FALSE)    
    }
    if(crop=="rice"){
        width=24
        height=13}else{
        width=14.3
        height=8.3}
        ggsave(violin,filename = paste(path2outputs,"/",crop,"_violin_plots.png",sep=""),width = width,height = height,units = "cm",dpi = 600)
    

#}    

[1] "buckwheat"


Warning message:
"Removed 7 rows containing non-finite values (stat_ydensity)."
Warning message:
"Removed 7 rows containing non-finite values (stat_boxplot)."


In [32]:
# For rice
width=24
height=13
ggsave(violin,filename = paste(path2outputs,"/",crop,"_violin_plots.png",sep=""),width = width,height = height,units = "cm",dpi = 600)

Warning message:
"Removed 7 rows containing non-finite values (stat_ydensity)."
Warning message:
"Removed 7 rows containing non-finite values (stat_boxplot)."


### Prepare maps

In [20]:
outer_border<-st_read(dsn=path2china,layer='CHN_adm0')

Reading layer `CHN_adm0' from data source `G:\My Drive\my_repositories\PhD\raw_data\CHN_adm' using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 70 fields
geometry type:  MULTIPOLYGON
dimension:      XY
bbox:           xmin: 73.5577 ymin: 18.15931 xmax: 134.7739 ymax: 53.56086
geographic CRS: WGS 84


In [34]:
crop="buckwheat"

In [35]:
posterior <-read.csv(paste(path2predictions,"/",crop,"_posterior_predictions_summary.csv",sep=""))

In [37]:
#macros<-loc@data[,c("Site","period","Sample_type",paste(crop,"Mean",sep=""))]
macros<-loc
colnames(macros@data)<-sub(paste(crop,"Mean",sep=""),"mean",colnames(macros@data))
macros<-macros[!is.na(macros$mean),]
macros@data$period<-factor(macros@data$period,levels=levels(macros@data$period)[levels(macros@data$period)<=max(macros$period)],ordered=TRUE)
slices<-seq(1000,as.numeric(gsub(" BP","",max(macros$period))),by=1000)
columns<-colnames(china@data)[unlist(lapply(paste("^",crop,"_mean_",slices,"_",sep=""),grep,colnames(china@data)))]
min<-floor(min(c(min(china@data[columns]),posterior$X5._log)))
max<-ceiling(max(c(max(china@data[columns])),posterior$X95._log))
scale_interval=5
if(crop=="wheat"){
            macros@data$map_labels <- ifelse(macros@data$period=="5000 BP"|
                                             macros@data$mean%in%unique(sort(macros@data$mean))[1:6]|
                                             (macros@data$period=="4000 BP"&coordinates(macros)[,"Longitude"]<90)|
                                             (macros$period %in% c("4000 BP")&macros$mean==min(macros[macros$period %in% c("4000 BP"),]$mean)),macros@data$Site,NA)
            macros@data$ymod <- ifelse(macros@data$map_labels %in% c("Donghuishan","Xishanping","Zhaojiazhuang"),0.5,ifelse(macros@data$map_labels=="Kyung-lung Mesa",0.9,ifelse(macros@data$map_labels%in%c("Yan\'erlong","Ashaonao","Xiaohe","Xintala"),0,-0.4)))
            macros@data$xmod <- ifelse(macros@data$map_labels =="Kyung-lung Mesa",0.4,ifelse(macros@data$map_labels %in% c("Ashaonao","Xiaohe","Xintala"),-1.5,ifelse(macros@data$map_labels=="Yan\'erlong",1.5,0)))        
            threshold<-unique(sort(macros$mean))[2]
}else if(crop=="barley"){
     macros@data$map_labels <- ifelse(macros@data$period=="5000 BP"| macros@data$mean%in%unique(sort(macros@data$mean))[1:5]|(macros@data$period=="4000 BP"&coordinates(macros)[,"Longitude"]<90),macros@data$Site,NA)
     macros@data$ymod <- ifelse(macros@data$map_labels %in% c("Donghuishan","Xishanping","Ding Dun"),0.5,ifelse(macros@data$map_labels =="Kyung-lung Mesa",0,-0.5))
     macros@data$xmod <- ifelse(macros@data$map_labels =="Kyung-lung Mesa",2.5,0)        
    threshold<-unique(sort(macros$mean))[2]
}else if(crop=="millet"){threshold<-unique(sort(macros$mean))[4]
                  macros@data$map_labels <- ifelse(macros@data$period>="8000 BP"| macros@data$mean%in%unique(sort(macros@data$mean))[1:4]|
                                                   (coordinates(macros)[,"Latitude"]>40&coordinates(macros)[,"Longitude"]>110)|
                                                   (coordinates(macros)[,"Latitude"]>40&coordinates(macros)[,"Longitude"]<90),macros@data$Site,NA)
                  macros@data$ymod <- ifelse(macros@data$map_labels %in%c("Peiligang","Xinglongwa","Dongkang","Luanzagangzi"),0.7,ifelse(macros@data$map_labels %in% c("Changguogou","Chengtoushan","Karuo","Shuiquan","Zhaobaogou","Xiaohe"),-0.7,ifelse(macros@data$map_labels%in%c("Yanghai","Yuergou"),0,-1.2)))
                  macros@data$xmod <- ifelse(macros@data$map_labels %in% c("Changguogou","Karuo"),-0.5,ifelse(macros@data$map_labels=="Yanghai",1.5,ifelse(macros@data$map_labels=="Yuergou",-1.5,0)))
                  macros@data$map_labels<-gsub(" ","\n",macros@data$map_labels)
                  macros$species<-ifelse(!is.na(macros$Foxtail.millet)&!is.na(macros$Broomcorn.millet),"both",
                        ifelse(!is.na(macros$Foxtail.millet),"foxtail",
                              ifelse(!is.na(macros$Broomcorn.millet),"broomcron","unidentified")))
}else if(crop=="rice"){
    threshold<-unique(sort(macros$mean))[1]
    macros@data$species<-ifelse(!is.na(macros$Rice),"rice","wild\nrice")
    macros@data$species<-ifelse(macros@data$period>"10000 BP","wild\nrice",macros@data$species)
    macros@data$legend_sizes<-ifelse(!is.na(macros$Rice),0.1,0.3)
    macros@data$map_labels <- ifelse(macros@data$period>"8000 BP"| macros@data$mean%in%unique(sort(macros@data$mean))[1]|
                                     (macros@data$period %in% c("7000 BP","8000 BP")&coordinates(macros)[,"Latitude"]>33.6)|
                                     (macros@data$period =="6000 BP"&coordinates(macros)[,"Latitude"]<28)|
                                     (macros@data$period =="4000 BP"&coordinates(macros)[,"Latitude"]>39),macros@data$Site,NA)
    macros@data$map_labels<-gsub(" ","\n",macros@data$map_labels)
    macros@data$ymod <- ifelse(macros@data$map_labels %in% c("Chenbeixi","Shangshan","Yuchanyan","Dingshishan"),-0.5,ifelse(macros@data$map_labels%in%c("Xianrendong","Yuezhuang"),-0.3,0.5))
    macros@data$xmod <- ifelse(macros@data$map_labels %in% c("Chenbeixi","Pengtoushan","Xianrendong","Yuchanyan","Yuezhuang","Dingshishan"),-1,1)
    scale_interval=10}
else if(crop=="buckwheat")

In [39]:
niche<-readOGR(paste(path2out,"/",crop,"_niche",sep=""),layer=paste(crop,"niche",sep="_"))
niche@data$facet<-factor(niche@data$facet,levels=unique(niche@data$facet),ordered=TRUE)

OGR data source with driver: ESRI Shapefile 
Source: "G:\My Drive\my_repositories\PhD\outputs\buckwheat_niche", layer: "buckwheat_niche"
with 8 features
It has 2 fields


In [ ]:
pred_maps<-tm_shape(china) +
    # Define the grid form maps drawing
    tm_graticules(lwd = 0.05,col="grey75",n.x=2,n.y=3,labels.size = 0.8)+
    # Define the colors of the polygons and the legend
    tm_fill(col=columns, title.col="Period",style="cont",breaks=seq(min,max,scale_interval),legend.reverse = TRUE,palette="-RdYlBu",title = paste("log area\nfraction\nunder\n",crop,"\ncultivation",sep=""),showNA = F,midpoint=threshold)+
    # Define the facets to draw:
    tm_facets(free.scales=FALSE,free.coords=FALSE,ncol = 2)+
    # Add innter boarder
    tm_shape(niche[niche@data$threshold=="maximum extent",])+
    tm_borders(col="grey10",lwd=1,lty="dashed")+
    tm_facets(by = "facet",free.scales=FALSE,free.coords=FALSE,ncol=2)+
    # Add innter boarder
    tm_shape(niche[niche@data$threshold=="extent by period",])+
    tm_borders(col="red",lwd=1,lty="dashed")+
    tm_facets(by = "facet",free.scales=FALSE,free.coords=FALSE,ncol=2,drop.empty.facets = FALSE)+
    # Add outer border:
    tm_shape(outer_border)+
    tm_borders(col="grey75",lwd=1)+
    # Format layout:
    tm_layout(panel.label.bg.color="white",
              frame=FALSE,
              legend.outside.size=0.08,
              legend.outside = TRUE,
              legend.text.size = 0.4,
              panel.label.size = 1,
              panel.label.fontface = "bold",
              legend.title.size=0.8,
              frame.lwd = NA,
              panel.label.height=2,
              attr.outside=TRUE,
              outer.margins=c(0.00001,0.01,0.01,0.01)
             )+
    tm_add_legend("line",labels=str_wrap(c("maximum extent","","","","","maximum extent up to time period","",""),7),col=c("grey10","white","white","white","white","red","white","white"),lty="dashed") 

In [29]:
if(crop%in%c("wheat","barley")){
    pred_maps<-pred_maps+
    # Add the location of macro and microfossils to the map
    tm_shape(macros)+
    tm_symbols(size=0.1,col="black",legend.col.show=FALSE,legend.shape.show=FALSE)+
    tm_text(text="map_labels",size=0.7,ymod="ymod",xmod="xmod")+
    tm_facets(by = "period",free.scales=FALSE,free.coords=FALSE,ncol=2,drop.empty.facets = FALSE)+
    tm_add_legend("symbol",labels=str_wrap(c("\n",paste(crop,"records\n"),"\n"),7),col=c("white","black","white"),size=0.3,shape=c(16,16,16))+
    tm_credits("Coordinate system: World Geodetic System 1984",position=c(0.05,0.7))
}else if(crop=="millet"){
    pred_maps<-pred_maps+
    # Add the location of macro and microfossils to the map
    tm_shape(macros)+
    tm_symbols(size=0.3,col ="species", shape="species", legend.col.show=FALSE,legend.shape.show=FALSE,palette=c("black","slateblue","mediumseagreen","white"),shapes=c(16,21,24,22),alpha=0.6)+
    tm_facets(by = "period",free.scales=FALSE,free.coords=FALSE,ncol=2,drop.empty.facets = FALSE)+
    tm_add_legend("symbol",labels=c("both","broomcorn","foxtail","unidentified"),col=c("black","slateblue","mediumseagreen","white"),shape=c(16,21,24,22),alpha=0.6)+
    tm_text(text="map_labels",size=0.7,xmod="xmod",ymod="ymod")+
    tm_credits("Coordinate system: World Geodetic System 1984",position=c(0.05,0.8))

}else if(crop=="rice"){
    set.seed(27271)
    pred_maps<-pred_maps+
    # Add the location of macro and microfossils to the map
    tm_shape(macros)+
    tm_symbols(size="legend_sizes",col ="species", shape="species", legend.col.show=FALSE,legend.shape.show=FALSE,palette=c("black","mediumseagreen"),shapes=c(16,22),legend.size.show=FALSE,alpha=0.6)+
    tm_facets(by = "period",free.scales=FALSE,free.coords=FALSE,ncol=2,drop.empty.facets = FALSE)+
    tm_add_legend("symbol",labels=c("rice","","wild\nrice"),col=c("black","white","mediumseagreen"),shape=c(16,16,22),size=c(0.5,1,0.7),alpha=0.6)+
    tm_text(text="map_labels",size=0.7,xmod="xmod",ymod="ymod")+
    tm_credits("Coordinate system: World Geodetic System 1984",position=c(0.05,0.8))

}

In [30]:
#width=17.4
#height=23.4
if (crop=="rice"){
    width=14.3
    height=37.24
    
}else if(crop=="millet"){
   width=14.3
   height=26.6
}else{
    width=14.3
    height=16
}
#width=14.3
#height=26.6
#height=38.24
tmap_save(tm = pred_maps,filename = paste(path2outputs,"\\",crop,"_predictions.png",sep=""),width = width,height = height,units = "cm",dpi = 600)

Warning message in sp::proj4string(obj):
"CRS object has comment, which is lost in output"
Warning message:
"Values have found that are higher than the highest break"
Some legend labels were too wide. These labels have been resized to 0.33, 0.33. Increase legend.width (argument of tm_layout) to make the legend wider and therefore the labels larger.

Some legend labels were too wide. These labels have been resized to 0.32, 0.30. Increase legend.width (argument of tm_layout) to make the legend wider and therefore the labels larger.

Map saved to G:\My Drive\my_repositories\PhD\manuscript\Chapter6\Figs\Raster\millet_predictions.png

Resolution: 3377.953 by 6283.465 pixels

Size: 5.629921 by 10.47244 inches (600 dpi)



In [26]:
pred_maps<-tm_shape(china) +
    # Define the grid form maps drawing
    tm_graticules(lwd = 0.05,col="grey75",n.x=2,n.y=3,labels.size = 0.8)+
    # Define the colors of the polygons and the legend
    tm_fill(col=columns, title.col="Period",style="cont",breaks=seq(min,max,5),legend.reverse = TRUE,palette="-RdYlBu",title = paste("log area\nfraction\nunder\n",crop,"\ncultivation",sep=""),showNA = F,midpoint=threshold)+
    # Define the facets to draw:
    tm_facets(free.scales=FALSE,free.coords=FALSE,ncol = 2)+
    # Add innter boarder
    tm_shape(niche[niche@data$threshold=="maximum extent",])+
    tm_borders(col="grey10",lwd=1,lty="dashed")+
    tm_facets(by = "facet",free.scales=FALSE,free.coords=FALSE,ncol=2)+
    # Add innter boarder
    tm_shape(niche[niche@data$threshold=="extent by period",])+
    tm_borders(col="red",lwd=1,lty="dashed")+
    tm_facets(by = "facet",free.scales=FALSE,free.coords=FALSE,ncol=2,drop.empty.facets = FALSE)+
    # Add outer border:
    tm_shape(outer_border)+
    tm_borders(col="grey75",lwd=1)+
    # Add the location of macro and microfossils to the map
    tm_shape(macros)+
    tm_symbols(size=0.1,col="black",legend.col.show=FALSE,legend.shape.show=FALSE)+
    tm_text(text="map_labels",size=0.7,ymod=-0.8,xmod=1)+
    tm_facets(by = "period",free.scales=FALSE,free.coords=FALSE,ncol=2,drop.empty.facets = FALSE)+
     # Format layout:
    tm_layout(panel.label.bg.color="white",
              frame=FALSE,
              legend.outside.size=0.08,
              legend.outside = TRUE,
              legend.text.size = 0.4,
              panel.label.size = 1,
              panel.label.fontface = "bold",
              legend.title.size=0.8,
              frame.lwd = NA,
              panel.label.height=2,
              attr.outside=TRUE
             )+
tm_add_legend("line",labels=str_wrap(c("maximum extent","","","","","maximum extent up to time period","",""),7),col=c("grey10","white","white","white","white","red","white","white"),lty="dashed")+
tm_add_legend("symbol",labels=str_wrap(c("\n",paste(crop,"records\n"),"\n"),7),col=c("white","black","white"),size=0.3,shape=c(16,16,16))+
tm_credits("\n\nCoordinate system: World Geodetic System 1984",position=c(0,0.5))